<a href="https://colab.research.google.com/github/ejjiang/ParselyCSV/blob/main/Generating_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating click tracking reports

## Enviroment set up (run once)

In [ ]:
# --- Install & Auth (run once)
!pip -q install gspread gspread-dataframe

from google.colab import auth
from google.colab import files
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import pandas as pd

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# --- Load a worksheet by URL
def load_sheet_private(sheet_url: str, worksheet: str|int|None=None) -> pd.DataFrame:
    """
    worksheet:
      - None -> first sheet
      - int  -> index (0-based)
      - str  -> tab name
    """
    sh = gc.open_by_url(sheet_url)
    if worksheet is None:
        ws = sh.get_worksheet(0)
    elif isinstance(worksheet, int):
        ws = sh.get_worksheet(worksheet)
    else:
        ws = sh.worksheet(worksheet)
    return pd.DataFrame(ws.get_all_records(numericise_ignore=['all']))

# --- Analysis (same as Option A)
def quick_analysis(df: pd.DataFrame, target: str|None=None, top_k: int=10):
    print("Rows, Columns:", df.shape)
    print("\nColumns & dtypes:")
    print(df.dtypes)
    print("\nMissing values:")
    print(df.isna().sum().sort_values(ascending=False).head(20))
    print("\nNumeric summary:")
    print(df.describe(include='number').T)

    cat_cols = [c for c in df.columns if df[c].dtype == 'object']
    if cat_cols:
        print("\nTop categories (first few object columns):")
        for c in cat_cols[:3]:
            print(f"\n[{c}] top values:")
            print(df[c].value_counts(dropna=False).head(top_k))

    num_cols = df.select_dtypes(include='number').columns
    if len(num_cols) >= 2:
        print("\nCorrelation (numeric):")
        print(df[num_cols].corr().round(3))

    if target and target in df.columns:
        if pd.api.types.is_numeric_dtype(df[target]):
            print(f"\nTarget '{target}' distribution:")
            print(df[target].describe())
        else:
            print(f"\nTarget '{target}' value counts:")
            print(df[target].value_counts(dropna=False).head(20))

In [ ]:
import pandas as pd
import re

def dedupe(cols):
    seen = {}
    out = []
    for c in cols:
        k = c if c else "unnamed"
        if k in seen:
            seen[k] += 1
            out.append(f"{k}_{seen[k]}")
        else:
            seen[k] = 0
            out.append(k)
    return out

def load_sheet_private(sheet_url: str, worksheet=None) -> pd.DataFrame:
    sh = gc.open_by_url(sheet_url)
    if worksheet is None:
        ws = sh.get_worksheet(0)
    elif isinstance(worksheet, int):
        ws = sh.get_worksheet(worksheet)
    else:
        ws = sh.worksheet(worksheet)

    # Get everything (raw values)
    values = ws.get_all_values()
    if not values:
        return pd.DataFrame()

    # First row as headers (deduped)
    headers = dedupe(values[0])
    df = pd.DataFrame(values[1:], columns=headers)

    # Optional: clean up blanks and try numeric conversion
    df = df.replace("", pd.NA)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="ignore")

    return df


## Import sheet via Google shared Links

In [ ]:
sheet_link = "https://docs.google.com/spreadsheets/d/1qhT__KhN3YquVDIc2xq_L3SSJ82EHpzTjlyxlUjQyC8/edit?gid=1125319047#gid=1125319047"
worksheet_name = "New With Link"
df = load_sheet_private(sheet_link, worksheet_name)
quick_analysis(df, target=None)

print("(rows,cols): ",df.shape)
df.head(10)     # first 10 rows


/tmp/ipython-input-1547264141.py:38: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


Rows, Columns: (14892, 98)

Columns & dtypes:
id                      object
external_message_id     object
from                    object
to                      object
cc                     float64
                        ...   
inferred_click_url      object
inferred_click_ts       object
step_1                 float64
variant_1              float64
delta_seconds          float64
Length: 98, dtype: object

Missing values:
cc                       14892
github_url               14892
facebook_url             14892
opened_at_1              14892
source_created_at        14888
source_updated_at        14888
replied_at               14885
linkedin_last_updated    14758
replied_at_1             14731
twitter_url              14682
bounced_at               14619
connections              14546
follower_count           14546
variant_1                14474
step                     14474
step_1                   14474
variant                  14474
scheduled_at             13963
error       

,id,external_message_id,from,to,cc,subject,body,opened_at,campaign_lead_id,created_at,...,variant,click_ts,delta_sec,track_clicks_1,email_1,inferred_click_url,inferred_click_ts,step_1,variant_1,delta_seconds
0,000c3853-5978-4138-a642-58d5ad19184d,<4886bd3d-ba8d-975e-09f6-c250d2a78c61@userlci....,brian@userlci.com,Laura Nagel <lnagel@sandburg.edu>,NaN,NHS Meeting Request,"<p>Hi Laura,<br><br>Are you or anyone from you...",<NA>,bead7126-8876-442b-b04f-1436468dacee,2025-03-15 11:22:18,...,NaN,<NA>,NaN,{},lnagel@sandburg.edu,<NA>,<NA>,NaN,NaN,NaN
1,16471579-5585-499d-8d30-de4253d783f8,<f5974d36-3447-0aa6-428c-9fc17ecf1138@userlci....,brianhansen@userlci.com,Brady Seavert <bradyseavert@bobcat.com>,NaN,Bring in firepower without bloating your team,"<p>Brady,</p><p></p><p>If your internal team i...",<NA>,589a32de-96e2-4a84-a863-911651e8494e,2025-06-09 20:08:45,...,NaN,<NA>,NaN,"{""https://www.rlci.com/work"": [{""ts"": ""2025-06...",bradyseavert@bobcat.com,<NA>,<NA>,NaN,NaN,NaN
2,2f3ae55d-3647-427d-9126-fb264e2570e1,<34bac638-679b-9266-7894-d95bc87e3e11@userlci....,hansen@userlci.com,Kenneth Scott <kenscott@kenscustomtees.com>,NaN,Can we help your business locally?,"<p>Hi Kenneth,</p><p></p><p>I’m Brian from <a ...",<NA>,9972ab2c-b18a-45b9-92df-fce8fc29fc57,2025-06-30 15:55:19,...,NaN,<NA>,NaN,{},kenscott@kenscustomtees.com,<NA>,<NA>,NaN,NaN,NaN
3,f5305af6-e819-46d1-9ac0-eedac56568ac,<NA>,<NA>,Kelly Ness <kness1@kish.edu>,NaN,Want a Free Campaign Concept?,"<p>Hi Kelly ,<br><br>Let’s make this easy: if ...",<NA>,70ea1677-c0de-4eb5-88a2-1441530a22ec,2025-07-25 19:43:25,...,NaN,<NA>,NaN,"{""https://www.rlci.com/"": [{""ts"": ""2025-07-25T...",kness1@kish.edu,<NA>,<NA>,NaN,NaN,NaN
4,f5305af6-e819-46d1-9ac0-eedac56568ac,<NA>,<NA>,Kelly Ness <kness1@kish.edu>,NaN,Want a Free Campaign Concept?,"<p>Hi Kelly ,<br><br>Let’s make this easy: if ...",<NA>,70ea1677-c0de-4eb5-88a2-1441530a22ec,2025-07-25 19:43:25,...,NaN,<NA>,NaN,"{""https://www.rlci.com/"": [{""ts"": ""2025-07-25T...",kness1@kish.edu,<NA>,<NA>,NaN,NaN,NaN
5,b91a0f8f-c169-4066-8f4e-ba057d463773,<c816c6d9-605c-ded3-b296-6c16e077cf94@userlci....,brian@userlci.com,Kelly Ness <kness1@kish.edu>,NaN,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Kelly ,</p><p></p><p>Red Letter has an ...",2025-07-25 19:43:41,70ea1677-c0de-4eb5-88a2-1441530a22ec,2025-03-18 11:21:51,...,1.0,2025-07-25 19:43:40.857 -0700,0.006,"{""https://www.rlci.com/"": [{""ts"": ""2025-07-25T...",kness1@kish.edu,https://www.rlci.com/,2025-07-25 19:43:40.857 -0700,0.0,1.0,0.006
6,b91a0f8f-c169-4066-8f4e-ba057d463773,<c816c6d9-605c-ded3-b296-6c16e077cf94@userlci....,brian@userlci.com,Kelly Ness <kness1@kish.edu>,NaN,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Kelly ,</p><p></p><p>Red Letter has an ...",2025-07-25 19:43:41,70ea1677-c0de-4eb5-88a2-1441530a22ec,2025-03-18 11:21:51,...,1.0,2025-07-25 19:43:40.857 -0700,0.006,"{""https://www.rlci.com/"": [{""ts"": ""2025-07-25T...",kness1@kish.edu,https://www.rlci.com/,2025-07-25 19:43:40.857 -0700,0.0,1.0,0.006
7,abd09c56-87c0-4b5d-b3d4-c3a6fab86e07,<3c4d4890-1368-6e51-e890-e611afe8b1ef@userlci....,brianhansen@userlci.com,Jennifer Ryan <jryan@fatbrands.com>,NaN,MTN West Loveland Event,"<p>Jennifer, will you or your team be attendin...",<NA>,fd531178-dfbb-49fc-804e-eaa245c8a19f,2025-03-14 21:39:04,...,NaN,<NA>,NaN,{},jryan@fatbrands.com,<NA>,<NA>,NaN,NaN,NaN
8,a7089204-5909-44fc-a6d1-fefce6824774,<ab7ab899-7573-ed35-8840-1b697773665e@userlci....,hansen@userlci.com,Laura Nagel <lnagel@sandburg.edu>,NaN,MTN West Loveland Event,"<p>Laura, will you or your team be attending M...",<NA>,bead7126-8876-442b-b04f-1436468dacee,2025-03-18 21:16:58,...,NaN,<NA>,NaN,{},lnagel@sandburg.edu,<NA>,<NA>,NaN,NaN,NaN
9,74a8a531-56af-43f9-b015-4fe6e4c723e0,<de2a622c-da50-0522-de7e-4355842dfddb@userlci....,hansen@userlci.com,Stephanie Haase <stephanie.haase@tunnellconsul...,NaN,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Stephanie ,</p><p></p><p>Red Letter has.

## Import sheet via CSV upload

In [ ]:
# Upload a CSV file from your local machine
uploaded = files.upload()

# Get the first uploaded file name
filename = list(uploaded.keys())[0]

# Read into pandas DataFrame
df = pd.read_csv(filename)

print(f"Loaded '{filename}' with {df.shape[0]} rows and {df.shape[1]} columns")
df.head()

IndexError: list index out of range

## Clicked msg & people analysis

In [ ]:
# get column headers
for col in df.columns:
    print(col)

id
external_message_id
from
to
cc
subject
body
opened_at
campaign_lead_id
created_at
updated_at
status
sent_at
organization_id
clicked_at
external_thread_id
headers
sequence_step_id
sequence_variant_id
replied_at
id_1
first_name
last_name
title
email
phone_numbers
employment_history
linkedin_slug
departments
functions
seniority
latest_employment_start_date
city
state
country
github_url
facebook_url
photo_url
twitter_url
account_name
lead_account_id
last_sync_at
created_at_1
updated_at_1
external_id
last_linkedin_sync_at
source_created_at
source_updated_at
email_status
education
accomplishment
activities
linkedin_last_updated
follower_count
connections
id_2
organization_id_1
opened_at_1
clicked_at_1
replied_at_1
campaign_id
created_at_2
updated_at_2
organization_lead_contact_id
sequence_completed
status_1
last_email_message_id
draft_email_message_id
error
job_id
deleted_at
scheduled_at
account_score
contact_score
score
max_score
track_clicks
track_opens
is_qualified
sender_email
first_s

In [ ]:
# convert columns to datetime
df["sent_at"] = pd.to_datetime(df["sent_at"], errors="coerce")
df["replied_at"] = pd.to_datetime(df["replied_at"], errors="coerce")
df["clicked_at"] = pd.to_datetime(df["clicked_at"], errors="coerce")

# now subtraction works
df["time_diff"] = df["clicked_at"] - df["sent_at"]

# optional: convert into hours/days
df["time_diff_hours"] = df["time_diff"].dt.total_seconds() / 3600
df["time_diff_days"] = df["time_diff"].dt.days

# Sort by time_diff (largest first)
df_sorted = df.sort_values(by="time_diff", ascending=False)

def make_linkedin_url(slug):
    if pd.isna(slug) or slug == "":
        return ""
    slug = str(slug).strip()
    # Only accept if it starts with a letter
    if re.match(r"^[A-Za-z]", slug):
        return f"https://www.linkedin.com/in/{slug}"
    return ""

df.loc[:, "lead_linkedin_URL"] = df["linkedin_slug"].apply(make_linkedin_url)

# Preview
df[["linkedin_slug", "lead_linkedin_URL"]].head(10)

,linkedin_slug,lead_linkedin_URL
0,lauramaly,https://www.linkedin.com/in/lauramaly
1,brady-seavert-2655a97,https://www.linkedin.com/in/brady-seavert-2655a97
2,kenneth-scott-4732183a,https://www.linkedin.com/in/kenneth-scott-4732...
3,kelly-ness-designs,https://www.linkedin.com/in/kelly-ness-designs
4,kelly-ness-2696aa21a,https://www.linkedin.com/in/kelly-ness-2696aa21a
5,kelly-ness-designs,https://www.linkedin.com/in/kelly-ness-designs
6,kelly-ness-2696aa21a,https://www.linkedin.com/in/kelly-ness-2696aa21a
7,jennifer-ryan-13210ba,https://www.linkedin.com/in/jennifer-ryan-13210ba
8,lauramaly,https://www.linkedin.com/in/lauramaly
9,stephanie-haase-038a424,https://www.linkedin.com/in/stephanie-haase-03...


In [ ]:
# define cutoff in minutes
cutoff_seconds = 10

# filter rows where time_diff > cutoff
trueMsgClicks_df = df[df["time_diff"] > pd.Timedelta(seconds=cutoff_seconds)]
print("Unique Msgs that Clicked:", len(filtered_df))

# preview
trueMsgClicks_df.sort_values(by="replied_at", ascending=False)
trueMsgClicks_df[["first_name","last_name","title", "email","subject","body", "sent_at", "clicked_at","replied_at", "time_diff"]].head(10)

Unique Msgs that Clicked: 79


,first_name,last_name,title,email,subject,body,sent_at,clicked_at,replied_at,time_diff
5,Kelly,Ness,Director Of Marketing And Advertising,kness1@kish.edu,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Kelly ,</p><p></p><p>Red Letter has an ...",2025-07-25 19:43:25,2025-07-25 19:43:41,NaT,0 days 00:00:16
6,Kelly,Ness,Director Of Marketing And Advertising,kness1@kish.edu,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Kelly ,</p><p></p><p>Red Letter has an ...",2025-07-25 19:43:25,2025-07-25 19:43:41,NaT,0 days 00:00:16
9,Stephanie,Haase,Marketing Manager,stephanie.haase@tunnellconsulting.com,Need Big-Campaign Looks Without Big-Agency Drama?,"<p>Hey Stephanie ,</p><p></p><p>Red Letter has...",2025-07-25 16:00:36,2025-07-25 16:00:55,NaT,0 days 00:00:19
15,Natalie,Flowers,Director of Digital Marketing,natalie.flowers@bdel.com,Quick Wins for Brands Like Yours,"<p>Hi Natalie ,<br><br>Not sure if your team’s...",2025-07-29 15:06:41,2025-07-29 15:07:06,NaT,0 days 00:00:25
24,Brady,Seavert,VP of Construction Sales,bradyseavert@bobcat.com,"We don't need marketing, All we need is Video","<p>Brady,</p><p><br>We hear this all time, so ...",2025-04-17 22:41:57,2025-04-17 22:42:25,NaT,0 days 00:00:28
41,Brady,Seavert,VP of Construction Sales,bradyseavert@bobcat.com,The Great American Outdoors,"<p>Brady,</p><p></p><p>Summer is here and ther...",2025-05-21 16:06:27,2025-05-21 16:07:11,NaT,0 days 00:00:44
43,Brady,Seavert,VP of Construction Sales,bradyseavert@bobcat.com,Redefining Toughness,"<p>Brady,</p><p></p><p>The “toughness culture”...",2025-04-29 17:51:59,2025-04-29 17:52:30,NaT,0 days 00:00:31
44,Brady,Seavert,VP of Construction Sales,bradyseavert@bobcat.com,Made in the USA,"<p>Brady,</p><p></p><p>Is your team looking at...",2025-05-14 23:21:39,2025-05-14 23:22:10,NaT,0 days 00:00:31
48,Daniel,"Latham, P.E.",President at Bloomsdale Excavating,djl@blex.com,Local Video Work,"<p>Daniel,</p><p></p><p>What video assets do y...",2025-05-27 20:11:01,2025-05-27 20:11:36,NaT,0 days 00:00:35
57,Daniel,"Latham, P.E.",President at Bloomsdale Excavating,djl@blex.com,Video package,"<p>Daniel,</p><p></p><p>Whether you only need ...",2025-06-12 17:17:03,2025-06-12 17:18:20,NaT,0 days 00:01:17


In [ ]:
from bs4 import BeautifulSoup

def clean_html_with_links(html):
    if pd.isna(html):
        return ""
    soup = BeautifulSoup(html, "html.parser")

    # replace <br> with newline
    for br in soup.find_all("br"):
        br.replace_with("\n")
    # insert newline before <p>
    for p in soup.find_all("p"):
        p.insert_before("\n")

    # replace anchor with "text (url)"
    for a in soup.find_all("a"):
        href = a.get("href")
        text = a.get_text(strip=True)
        a.replace_with(f"{text} ({href})")

    return soup.get_text().strip()

# Apply to your DataFrame column
trueMsgClicks_df.loc[:, "body_clean"] = trueMsgClicks_df["body"].apply(clean_html_with_links)

In [ ]:
# ✏️ Choose the columns you want here
selected_cols = ["first_name","last_name","title","lead_linkedin_URL","email","subject","body_clean", "sent_at", "clicked_at"]

# Create a new dataframe with just those columns
df_export = trueMsgClicks_df[selected_cols].copy()
df_export = df_export.sort_values(by="first_name", ascending=True).reset_index(drop=True)

### Export to Excel

In [ ]:
# Save both DataFrames into Excel with multiple sheets
export_filename = "Click_tracking_analysis.xlsx"
with pd.ExcelWriter(export_filename, engine="openpyxl") as writer:
    people_counts.to_excel(writer, sheet_name="Clicked Leads Details", index=False)
    df_export.to_excel(writer, sheet_name="Clicked Msg Details", index=False)

# Open workbook with openpyxl for formatting
wb = load_workbook(export_filename)

# Format each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]

    # Set column widths + wrap/align
    for col in ws.columns:
        col_letter = col[0].column_letter
        ws.column_dimensions[col_letter].width = 20

        for cell in col:
            cell.alignment = Alignment(
                wrapText=False,
                vertical="center",
                horizontal="left"
            )

    # Row height
    for row in ws.iter_rows():
        ws.row_dimensions[row[0].row].height = 30

# Save formatted file
wb.save(export_filename)

# Download in Colab
files.download(export_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>